In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, utils

from skimage import io, transform

import os

from project_code.utils import preprocessing
from project_code.data.face_dataset import *

In [2]:
# Setup tensorboard
import tensorboard

%load_ext tensorboard
%tensorboard --logdir runs

# If you run this notebook locally, you can also access Tensorboard at 127.0.0.1:6006 now.

# Clean up old logs
if os.path.isdir('./runs/'):
    import shutil
    shutil.rmtree('runs/')

from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs"
writer = SummaryWriter('runs')

Reusing TensorBoard on port 6006 (pid 4699), started 0:02:53 ago. (Use '!kill 4699' to kill it.)

In [10]:
# Preprocessing.
if False:
    attributes = pd.read_csv('data/person.csv', sep=';')
    
    # Remove all people without pictures.
    with_images, without_images = preprocessing.filter_without_images(attributes, 'data/front/front/')

    without_images.to_csv("attributesPersonsNoImages.csv", index=False)
    with_images.to_csv("attributesPersonsWithImages.csv", index=False)

    # Use this when the people without images are already filtered out.
    # (Which should be the case if you have the csv files)
    #attributes = pd.read_csv("attributesPersonsWithImages.csv")
    
    train_set, val_set, test_set = preprocessing.split_data(with_images, 0.8, 0.05)

    train_set.to_csv("trainSet.csv", index=False)
    val_set.to_csv('valSet.csv', index=False)
    test_set.to_csv("testSet.csv", index=False)

train_set = pd.read_csv("trainSet.csv")
val_set = pd.read_csv("valSet.csv")
test_set = pd.read_csv("testSet.csv")

In [6]:
from project_code.networks.congregated_layers import *

# Define a convolutional neural network
class UNet(nn.Module):
    def __init__(self, bilinear):
        super(UNet, self).__init__()
        
        self.n_channels = 3
        self.bilinear = bilinear
        
        factor = 2 if bilinear else 1

        self.inc = DoubleConv(self.n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = nn.Conv2d(64, self.n_channels, kernel_size=1)
    
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

In [16]:
# Load the data.
train_set = FaceDataset(csv_file="trainSet.csv",
                        root_dir='data/front/front',
                        transform=transforms.Compose([
                            transforms.Resize((48, 48)),
                            transforms.ToTensor()
                        ]))

val_set = FaceDataset(csv_file="valSet.csv",
                      root_dir='data/front/front',
                      transform=transforms.Compose([
                          transforms.Resize((48, 48)),
                          transforms.ToTensor()
                      ]))

test_set = FaceDataset(csv_file="testSet.csv",
                       root_dir='data/front/front',
                       transform=transforms.Compose([
                           transforms.Resize((48, 48)),
                           transforms.ToTensor()
                       ]))

trainloader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=0)
valloader = DataLoader(val_set, batch_size=4, shuffle=True, num_workers=0)

# Show the data in Tensorboard
dataiter = iter(trainloader)
data = dataiter.next()
img_grid = utils.make_grid(data['image'])
writer.add_image('dataset_images', img_grid)

# Keeps track of how often we train the model,
# this way we will see the loss logs in different plots.
n_runs = 0

In [20]:
net = UNet(bilinear=True)

criterion = nn.MSELoss() # average loss over the whole reconstructed image.
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Add a scheme of our network to Tensorboard.
writer.add_graph(net.cpu(), data['image'])
writer.close()

writer = SummaryWriter('runs/{}'.format(n_runs))

# Train the network
for epoch in range(1):
    running_loss = 0.0
    valiter = iter(valloader)
    for i, data in enumerate(trainloader, start=0):
        inputs = data['image']
        labels = data['image']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Log the loss to Tensorboard,
        # giving a nice loss over time.
        writer.add_scalar('Loss/train', 
                          loss.item(), 
                          epoch * len(trainloader) + i)

        if i % 10 == 0:
            # Get images and swap axes.
            image_output = outputs[0].detach().numpy().T
            image_output = np.swapaxes(image_output, 0, 1)
            image_input = labels[0].detach().numpy().T
            image_input = np.swapaxes(image_input, 0, 1)

            # Add a comparison between input and output image to Tensorboard.
            fig, (ax1, ax2) = plt.subplots(1, 2)
            ax1.imshow(image_input)
            ax2.imshow((image_output * 255).astype(np.uint8))

            writer.add_figure('input vs. output',
                              fig,
                              global_step=epoch * len(trainloader) + i)
            
            # For now do the validation at the same frequency as the image feedback.
            val_data = valiter.next()
            val_inputs = val_data['image']
            val_outputs = net(val_inputs)
            
            val_loss = criterion(val_outputs, val_inputs)
            
            # Write the validation loss to Tensorboard.
            writer.add_scalar('Loss/val',
                              val_loss.item(),
                              epoch * len(trainloader) + i)
            
            # Instead of plotting training and validation loss on one graph,
            # create a third graph that shows the divergence.
            writer.add_scalar('Loss/divergence',
                              val_loss.item() - loss.item(),
                              epoch * len(trainloader) + i)

        if i == 100:
            break

n_runs = n_runs + 1
print('Finished training')

Finished training


In [9]:
# Show the feature maps for the latent space.
# Probably we want to either change either
# the latent space itself or the visualisation.

# Calling 'data' on weights makes
# a copy of the tensor for local use.
latent_conv_weights = net.down4.maxpool_conv[1].double_conv[3].weight.data

for i, kernel in enumerate(latent_conv_weights[0]):
    fig, ax1 = plt.subplots(1, 1)
    ax1.imshow((kernel.numpy()))
    writer.add_figure('Latent space weights',
                      fig,
                      global_step=i)

In [10]:
writer.close()
writer.flush()